## References

- [RDKit Documentation](https://www.rdkit.org/docs/GettingStartedInPython.html)
- [PubChem](https://pubchem.ncbi.nlm.nih.gov/) - find SDF files for molecules
- YouTube Tutorial
  - [Part1](https://www.youtube.com/watch?v=NozaWUkJ3YM)


## Init Parameters


In [1]:
from rdkit import Chem
from rdkit.Chem import Draw

from src import SDF_DIR

file = "chalconatronate.sdf"

## Sandbox


In [2]:
# Loads Molecule from file to python object (Properties in the SDF file are used to set properties on each molecule)
suppl: Chem.SDMolSupplier = Chem.SDMolSupplier(f"{SDF_DIR.joinpath(file)}")

# Test if each molecule was loadded correctly
for molecule in suppl:
    assert molecule is not None

mols = [x for x in suppl]  # one line for loop syntax, must know, very handy
print(f"The file '{file}' contains '{len(mols)}' molecule(s).")

assert len(mols) > 0  # make sure the file was loaded correctly


The file 'chalconatronate.sdf' contains '8' molecule(s).


#### 1. Compare RDkit with Element class. Preliminary coding.


In [3]:
# Calling some basic functions on atoms objects
from rdkit.Chem import rdMolDescriptors
from rdkit import Chem
from rdkit.Chem import rdchem

for i in range(len(mols)):  # Infromation about given molecule
    mol = mols[i]
    print(f"I am molecule {i + 1}")

    rdMolDescriptors.CalcOxidationNumbers(mol)

    mol = Chem.AddHs(mol)
    atoms: list[Chem.Atom] = [
        a for a in mol.GetAtoms()
    ]  # Creates a list of atoms for each molecule

    # Assign charge taken from SDF file ONLY for atoms without covalent bonds
    for atom in atoms:
        covalent_bond = (
            atom.GetTotalDegree()
        )  # Number of covalent bonds including implicit Hs
        if covalent_bond == 0:
            atom_charge = atom.GetFormalCharge()
        else:
            atom_charge = None

    ox_state_atoms = [
        "As",
        "Hg",
        "Pb",
    ]

    for atom in atoms:
        if (
            atom.HasProp("OxidationNumber")
            and atom_charge is None
            and atom.GetSymbol() in ox_state_atoms
        ):
            ox_state = atom.GetProp("OxidationNumber")
        else:
            ox_state = None

        neighbors = [nbr.GetSymbol() for nbr in atom.GetNeighbors()]
        neighbor_symbols = ", ".join(neighbors) if neighbors else "none"

    ring_atoms = [
        "N",
        "C",
    ]

    for atom in atoms:
        if atom.GetSymbol() in ring_atoms:
            in_ring = atom.IsInRing()
        else:
            in_ring = None

        print(
            f"""
            Atom: {atom.GetSymbol()},  
            IsInRing: {atom.IsInRing()},
            in_ring: {in_ring},
            GetNoBondedAtoms: {atom.GetTotalDegree()},
            FormalCharge: {atom.GetFormalCharge()},
            atom_charge: {atom_charge}
            ox_state: {ox_state}, 
            id: {atom.GetIdx()}, 
            neighbors: {neighbor_symbols}, 
            """
        )

I am molecule 1

            Atom: Na,  
            IsInRing: False,
            in_ring: None,
            GetNoBondedAtoms: 1,
            FormalCharge: 2,
            atom_charge: None
            ox_state: None, 
            id: 0, 
            neighbors: Na, 
            

            Atom: H,  
            IsInRing: False,
            in_ring: None,
            GetNoBondedAtoms: 1,
            FormalCharge: 0,
            atom_charge: None
            ox_state: None, 
            id: 1, 
            neighbors: Na, 
            
I am molecule 2

            Atom: Na,  
            IsInRing: False,
            in_ring: None,
            GetNoBondedAtoms: 0,
            FormalCharge: 1,
            atom_charge: 1
            ox_state: None, 
            id: 0, 
            neighbors: none, 
            
I am molecule 3

            Atom: Cu,  
            IsInRing: False,
            in_ring: None,
            GetNoBondedAtoms: 0,
            FormalCharge: 2,
            atom_charg

In [4]:
from constants import PASCAL_CONST

sum_dia_contr = 0

for i in range(len(mols)):
    mol = mols[i]

    print(f"I am molecule {i + 1}")

    rdMolDescriptors.CalcOxidationNumbers(mol)

    mol = Chem.AddHs(mol)
    atoms: list[Chem.Atom] = [
        a for a in mol.GetAtoms()
    ]  # Creates a list of atoms for each molecule

    for atom in atoms:
        symbol = atom.GetSymbol()
        if symbol in PASCAL_CONST:
            # retrieve const data
            covalent_data = PASCAL_CONST[symbol]["covalent"]
            ionic_data = PASCAL_CONST[symbol]["ionic"]["charge"]
            ox_state_data = PASCAL_CONST[symbol]["covalent"]["ox_state"]

            if in_ring:
                sum_dia_contr += covalent_data["ring"]

            """TODO: Resolve the error"""
            if not in_ring:
                sum_dia_contr += covalent_data["open_chain"]

            """TODO: Fix incompatible format of oxidation state. Ox_state from RDKit: int, Ox_state in PASCAL_CONST: string."""
            # if ox_state is not None:
            # ....

            """TODO: Fix incompatible format of charge. Charge from RDKit: int; Charge in PASCAL_CONST: string."""
            # if atom_charge is not None:
            # ....
    print(sum_dia_contr)

I am molecule 1
-12.129999999999999
I am molecule 2
-21.33
I am molecule 3


TypeError: unsupported operand type(s) for +=: 'float' and 'NoneType'